In [27]:
import requests
from bs4 import BeautifulSoup
from newspaper import Article
import json


In [28]:
def scrape_cnn_article(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    title = soup.find('h1').get_text() if soup.find('h1') else 'N/A'
    author = soup.find('span', {"class": 'byline__name'}).get_text() if soup.find('span', {"class": 'byline__name'}) else 'N/A'
    date = soup.find('div', {'class': 'timestamp'}).get_text() if soup.find('div', {'class': 'timestamp'}) else 'N/A'
    body = ' '.join([p.get_text() for p in soup.find_all('div', {'class': 'article__content-container'})])
    
    return {
        'title': title.strip(),
        'author': author.strip(),
        'date': date.strip(),
        'body': body.strip(),
        'url': url.strip()
    }


In [29]:
url = 'https://edition.cnn.com/business/tech'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
links = []
for link in  soup.findAll('a', {'class': 'container__link container__link--type-article container_vertical-strip__link'}):
    links.append(link['href'])

In [30]:
links = list(set(links))

In [32]:
all_articles_data = []
for i in links:
    url = f'https://edition.cnn.com{i}'
    article_data = scrape_cnn_article(url)
    all_articles_data.append(article_data)
with open('cnn_articles.json', 'w') as f:
    json.dump(all_articles_data, f,indent=4)


In [6]:
import json
from transformers import pipeline
import re

In [7]:
with open('cnn_articles.json', 'r') as file:
    original_json = json.load(file)


In [13]:
def extract_labels(articles):
    labels = set()
    for article in articles:
        words = re.findall(r'\b\w+\b', article['title'])
        for word in words:
            labels.add(word)
    return list(labels)

In [15]:
candidate_labels = extract_labels(original_json)

In [16]:
classifier = pipeline('zero-shot-classification', model='facebook/bart-large-mnli')

c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:982: UserWarning: Not enough free disk space to download the file. The expected file size is: 1629.44 MB. The target location C:\Users\HP\.cache\huggingface\hub\models--facebook--bart-large-mnli\blobs only has 1287.78 MB free disk space.
  warnings.warn(
c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:982: UserWarning: Not enough free disk space to download the file. The expected file size is: 1629.44 MB. The target location C:\Users\HP\.cache\huggingface\hub\models--facebook--bart-large-mnli\blobs only has 0.00 MB free disk space.
  warnings.warn(


ValueError: Could not load model facebook/bart-large-mnli with any of the following classes: (<class 'transformers.models.auto.modeling_auto.AutoModelForSequenceClassification'>, <class 'transformers.models.bart.modeling_bart.BartForSequenceClassification'>). See the original errors:

while loading with AutoModelForSequenceClassification, an error is thrown:
Traceback (most recent call last):
  File "c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\pipelines\base.py", line 283, in infer_framework_load_model
    model = model_class.from_pretrained(model, **kwargs)
  File "c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\models\auto\auto_factory.py", line 563, in from_pretrained
    return model_class.from_pretrained(
  File "c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\modeling_utils.py", line 3351, in from_pretrained
    resolved_archive_file = cached_file(pretrained_model_name_or_

In [ ]:
new_json = {
    "documents": []
}

# Classify each article and populate the new JSON structure
for article in original_json['articles']:
    # Join the title and body to create a single text for classification
    text = article['title'] + " " + " ".join(article['body'])
    
    # Perform the classification
    classification = classifier(text, candidate_labels)
    
    # Get the highest scoring category
    topic = classification['labels'][0]
    
    # Create the concept entry
    concept = {
        "concept": article['title'],
        "content": " ".join(article['body'])  # Join all paragraphs into a single content string
    }
    
    # Check if the topic already exists in new_json
    topic_exists = False
    for document in new_json["documents"]:
        if document["topic"] == topic:
            document["concepts"].append(concept)
            topic_exists = True
            break
    
    # If the topic does not exist, create a new topic entry
    if not topic_exists:
        new_json["documents"].append({
            "topic": topic,
            "concepts": [concept]
        })

# Convert the new structure to JSON
new_json_str = json.dumps(new_json, indent=4)

# Save the new JSON structure to a file
with open('converted_cnn_articles.json', 'w') as file:
    file.write(new_json_str)

print("Conversion complete. New JSON structure saved to 'converted_cnn_articles.json'.")


In [ ]:
import json
from transformers import pipeline
import re

# Load the original JSON file
with open('cnn_articles.json', 'r') as file:
    original_json = json.load(file)

# Extract candidate labels from article titles
def extract_labels(articles):
    labels = set()
    for article in articles:
        # Use simple regex to extract potential labels from titles
        # Here, we assume words in titles might be good labels
        words = re.findall(r'\b\w+\b', article['title'])
        for word in words:
            labels.add(word)
    return list(labels)

# Extract candidate labels from the articles
candidate_labels = extract_labels(original_json['articles'])

# Initialize the text classification pipeline
classifier = pipeline('zero-shot-classification', model='facebook/bart-large-mnli')

# Create a new JSON structure
new_json = {
    "documents": []
}

# Classify each article and populate the new JSON structure
for article in original_json['articles']:
    # Join the title and body to create a single text for classification
    text = article['title'] + " " + " ".join(article['body'])
    
    # Perform the classification
    classification = classifier(text, candidate_labels)
    
    # Get the highest scoring category
    topic = classification['labels'][0]
    
    # Create the concept entry
    concept = {
        "concept": article['title'],
        "content": " ".join(article['body'])  # Join all paragraphs into a single content string
    }
    
    # Check if the topic already exists in new_json
    topic_exists = False
    for document in new_json["documents"]:
        if document["topic"] == topic:
            document["concepts"].append(concept)
            topic_exists = True
            break
    
    # If the topic does not exist, create a new topic entry
    if not topic_exists:
        new_json["documents"].append({
            "topic": topic,
            "concepts": [concept]
        })

# Convert the new structure to JSON
new_json_str = json.dumps(new_json, indent=4)

# Save the new JSON structure to a file
with open('converted_cnn_articles.json', 'w') as file:
    file.write(new_json_str)

print("Conversion complete. New JSON structure saved to 'converted_cnn_articles.json'.")
